The ENSO Analyzer

This tool calculates the various components of the Bjerknes stability index for a given CMIP6 model + experiment combination. The calculation requires the following variables be availabe:


thetao (ocean potential temperature)
uo (ocean eastward velocity)
vo (ocean northward velocity)
wo (ocean upward velocity)
tauu (eastward surface wind stress)
tauv (northward surface wind stress)
hfls (surface upward latent heat)
hfss (surface upward sensible heat)
rlds (surface downwelling longwave flux)
rlus (surface upwelling longwave flux)
rsds (surface downwelling shortwave flux)
rsus (surface upwelling shortwave flux)

First, we load the modules to use and the cluster

In [9]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs

from dask_gateway import Gateway
from dask.distributed import Client

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.scale(20)
client = Client(cluster,timeout="120s")
cluster




Now we load all the CMIP6 datasets. I chose 200 year max to make the calculations faster. This can be modified

In [10]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

model_name='"MIROC6"'
experiment='"abrupt-4xCO2"'

test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

theta = df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id =='+ experiment+'& grid_label == "gn"')

uo = df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Omon" & variable_id == "uo" & experiment_id =='+ experiment)
vo = df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Omon" & variable_id == "vo" & experiment_id =='+ experiment)
wo = df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Omon" & variable_id == "wo" & experiment_id =='+ experiment)

tauu=df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)
tauv=df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

hfls=df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Amon" & variable_id == "hfls" & experiment_id =='+ experiment)
hfss=df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Amon" & variable_id == "hfss" & experiment_id =='+ experiment)
rlds=df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Amon" & variable_id == "rlds" & experiment_id =='+ experiment)
rlus=df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Amon" & variable_id == "rlus" & experiment_id =='+ experiment)
rsds=df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Amon" & variable_id == "rsds" & experiment_id =='+ experiment)
rsus=df.query('source_id =='+ model_name +'& activity_id=="CMIP" & table_id == "Amon" & variable_id == "rsus" & experiment_id =='+ experiment)


def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[-1]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

theta_pi=get_df(theta)
uo_pi=get_df(uo)
vo_pi=get_df(vo)
wo_pi=get_df(wo)
tauu_pi=get_df(tauu)
tauv_pi=get_df(tauv)
hfls_pi=get_df(hfls)
hfss_pi=get_df(hfss)
rlds_pi=get_df(rlds)
rlus_pi=get_df(rlus)
rsds_pi=get_df(rsds)
rsus_pi=get_df(rsus)

tm=12*200;

theta_pi=theta_pi.isel(time=slice(0,tm),lev=slice(0,15))
theta_pi_deep=theta_pi.isel(time=slice(0,tm))

uo_pi=uo_pi.isel(time=slice(0,tm),lev=slice(0,10))
vo_pi=vo_pi.isel(time=slice(0,tm),lev=slice(0,10))
wo_pi=wo_pi.isel(time=slice(0,tm),lev=slice(0,10))
tauu_pi=tauu_pi.isel(time=slice(0,tm))
tauv_pi=tauv_pi.isel(time=slice(0,tm))
hfls_pi=hfls_pi.isel(time=slice(0,tm))
hfss_pi=hfss_pi.isel(time=slice(0,tm))
rlds_pi=rlds_pi.isel(time=slice(0,tm))
rlus_pi=rlus_pi.isel(time=slice(0,tm))
rsds_pi=rsds_pi.isel(time=slice(0,tm))
rsus_pi=rsus_pi.isel(time=slice(0,tm))


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:463: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarray.py:102: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


To get ready to perform the analysis, I like to regrid the ocean data, to make it easier to work with. Note, this section may have to be modified depending on which model you use.

In [9]:
u_bar=uo_regrid.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).isel(lev=slice(0,10)).mean('lon').mean('lat')
u_bar=(u_bar*zz).sum('lev')/sum(zz)
v_bar=vo_regrid.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).isel(lev=slice(0,10)).mean('lon').mean('lat')
v_bar=(v_bar*zz).sum('lev')/sum(zz)
w_bar=wo_regrid.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).isel(lev=slice(0,10)).mean('lon').mean('lat')
w_bar=(w_bar*zz).sum('lev')/sum(zz)


In [11]:

import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

#xesmf doesn't like it when I define the grid from the dataset, so I like to explicitly define input and output grids
# For a curvelinear grid, we need both lat/lon and lat/lon bounds

lonO=theta_pi.longitude
latO=theta_pi.latitude
lonO_b=theta_pi.x_bnds
latO_b=theta_pi.y_bnds

grid_in = {'lon': lonO, 'lat': latO,
           'lon_b': lonO_b, 'lat_b': latO_b} 


grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 90, 1.0)),
                     'lon': (['lon'], np.arange(0, 359, 1)),})
                       
regridder = xe.Regridder(theta_pi, grid_out, 'bilinear')

thetao_regrid = regridder(theta_pi['thetao'])
thetao_deep_regrid = regridder(theta_pi_deep['thetao'])
uo_regrid = regridder(uo_pi['uo']) 
vo_regrid = regridder(vo_pi['vo'])  
wo_regrid = regridder(wo_pi['wo'])  
                       

/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:439: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:439: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:439: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:439: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply

Now it's time to calculate the parameters that we need. 

In [12]:
tauu_pi_clim=tauu_pi.groupby('time.month')-tauu_pi.groupby('time.month').mean('time')

eq_zo_wind=tauu_pi_clim['tauu'].sel(lon=slice(150,270)).sel(lat=slice(-5,5)).mean('lon').mean('lat')

clim=thetao_regrid.groupby('time.month').mean('time')
#clim=clim.chunk(6, 15, 180, 359)
thetao_regrid_clim=thetao_regrid.groupby('time.month')-clim

#thetao_regrid_clim=thetao_regrid_clim.chunk(6, 15, 180, 359)

nino_temp=thetao_regrid_clim.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).isel(lev=0).mean('lon').mean('lat')

Q_pi=(rsds_pi['rsds']-rsus_pi['rsus'])-(rlus_pi['rlus']-rlds_pi['rlds'])-hfls_pi['hfls']-hfss_pi['hfss']
Q_pi_clim=Q_pi.groupby('time.month')-Q_pi.groupby('time.month').mean('time')
nino_Q=Q_pi_clim.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).mean('lon').mean('lat')

zz=np.empty((len(uo_pi.lev)))
zz[0]=1
for i in range(1,len(uo_pi.lev)-1):
    zz[i+1]=uo_pi.lev[i+1]-uo_pi.lev[i]

uo_regrid_clim=uo_regrid.groupby('time.month')-uo_regrid.groupby('time.month').mean('time')
ux=(uo_regrid_clim.sel(lon=slice(170,240)).sel(lat=slice(-5,5))).mean('lon').mean('lat')
ux=(ux*zz).sum('lev')/sum(zz)

wo_regrid=wo_regrid.where(wo_regrid.mean('time')>0)
wo_regrid_clim=wo_regrid.groupby('time.month')-wo_regrid.groupby('time.month').mean('time')
#wo_regrid_clim=wo_regrid_clim.chunk()
wx=(wo_regrid.sel(lon=slice(170,240)).sel(lat=slice(-5,5))).mean('time')
wx_pos=(wx.sel(lon=slice(170,240)).sel(lat=slice(-5,5))).mean('lon').mean('lat')
wx_pos=(wx_pos*zz).sum('lev')/sum(zz)

mask1=  ~np.isnan(wx_pos)

h1=thetao_deep_regrid['lev'].where(thetao_regrid > 20)
h1=h1.where(thetao_regrid < 22)
h=h1.mean(dim="lev")
h_clim=h.groupby('time.month')-h.groupby('time.month').mean('time')
h_nino=h_clim.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).mean('lon').mean('lat')

T_50=thetao_regrid_clim.isel(lev=slice(0,10))
wo_50=wo_regrid.isel(lev=5).mean('time')
T_50_pos=T_50.where(wo_50>0)
T_50_nino=T_50_pos.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).mean('lon').mean('lat')
T_50_nino=(T_50_nino*zz).sum('lev')/sum(zz)
mask2=  ~np.isnan(T_50_nino)


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 200 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 67 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 19 times more chunks
  return self.array[key]


In [15]:

u_bar=uo_regrid.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).isel(lev=slice(0,10)).mean('lon').mean('lat')
u_bar=(u_bar*zz).sum('lev')/sum(zz)
v_bar=vo_regrid.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).isel(lev=slice(0,10)).mean('lon').mean('lat')
v_bar=(v_bar*zz).sum('lev')/sum(zz)
w_bar=wo_regrid.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).isel(lev=slice(0,10)).mean('lon').mean('lat')
w_bar=(w_bar*zz).sum('lev')/sum(zz)

sst_mean=thetao_regrid.sel(lat=slice(-5,5)).isel(lev=0).mean('time')
sst_mean.load()
ml_mean=thetao_regrid.sel(lon=slice(170,240),lat=slice(-5,5)).isel(lev=slice(0,10)).mean('time').mean('lon').mean('lat')
ml_mean.load()

dT_dx=xr.full_like(sst_mean, np.nan, dtype=np.double)
dT_dz=xr.full_like(ml_mean, np.nan, dtype=np.double)
                                    
import xarray.ufuncs as xu
from pylab import *

#dT_dx
dT_dx[:,0]=(sst_mean.isel(lon=0)-sst_mean.isel(lon=len(sst_mean.lon)-1))/(110000*xu.cos(sst_mean.lat*0.0174533))
for i in range(1,len(sst_mean.lon)-1):
        dT_dx[:,i]=(sst_mean.isel(lon=i+1)-sst_mean.isel(lon=i))/(110000*xu.cos(sst_mean.lat*0.0174533))

#dT_dz
for i in range(1,len(ml_mean.lev)):
        dT_dz[i]=(ml_mean.isel(lev=i)-ml_mean.isel(lev=i-1))

        
dT_dx_nino=dT_dx.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).mean('lon').mean('lat')
dT_dz_nino=(dT_dz*zz).sum('lev')/sum(zz)

dT_dx_nino.to_netcdf('MIROC6/4xCO2/dT_dx_nino.nc')
dT_dz_nino.to_netcdf('MIROC6/4xCO2/dT_dz_nino.nc')  

KeyboardInterrupt: 

In [ ]:
eq_zo_wind.load()
eq_zo_wind.to_netcdf('MIROC6/4xCO2/eq_zo_wind.nc')

Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.8/ssl.py", line 1309, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1124)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 696, in _handle_events
    self._handle

In [7]:

nino_temp.load()
nino_temp.to_netcdf('MIROC6/4xCO2/nino_temp.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [6]:
nino_Q.load()
nino_Q.to_netcdf('MIROC6/4xCO2/nino_Q.nc')

In [ ]:
ux.load()
ux.to_netcdf('ux.nc')

Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.8/ssl.py", line 1309, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1124)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 696, in _handle_events
    self._handle

In [6]:
wx_pos.load()
wx_pos.to_netcdf('wx_pos.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [7]:
h_nino.load()
h_nino.to_netcdf('MIROC6/4xCO2/h_nino_1.nc')

distributed.client - ERROR - Failed to reconnect to scheduler after 50.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


CancelledError: ('mean_agg-aggregate-49a911f517be003c5ad7edf94186fe21', 307)

Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.8/ssl.py", line 1309, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1124)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 696, in _handle_events
    self._handle

In [ ]:
T_50_nino.load()
T_50_nino.to_netcdf('MIROC6/4xCO2/T_50_nino.nc')

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [15]:
u_bar.load()
u_bar.to_netcdf('MIROC6/4xCO2/u_bar.nc')

In [16]:
v_bar.load()
v_bar.to_netcdf('MIROC6/4xCO2/v_bar.nc')

In [10]:
w_bar.load()
w_bar.to_netcdf('MIROC6/4xCO2/w_bar.nc')

In [79]:
eq_zo_wind.to_netcdf('MIROC6/4xCO2/eq_zo_wind.nc')

In [10]:
dT_dx_nino=dT_dx.sel(lon=slice(170,240)).sel(lat=slice(-5,5)).mean('lon').mean('lat')
dT_dz_nino=dT_dz.mean('lev')

dT_dx_nino.to_netcdf('MIROC6/dT_dx_nino.nc')
dT_dz_nino.to_netcdf('MIROC6/dT_dz_nino.nc')  

DataArrayGroupBy, grouped over 'month' 
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [13]:
x=thetao_regrid.groupby('time.month').mean('time')
x

<xarray.DataArray 'thetao' (month: 12, lev: 15, lat: 180, lon: 359)>
dask.array<stack, shape=(12, 15, 180, 359), dtype=float64, chunksize=(1, 15, 180, 359), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 1.0 3.5 7.0 11.0 15.5 ... 54.0 62.0 70.0 78.0 87.0
  * lon      (lon) int64 0 1 2 3 4 5 6 7 8 ... 351 352 353 354 355 356 357 358
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 86.0 87.0 88.0 89.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12